In [109]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import datetime

In [110]:
service_request = pd.read_csv('../data/hubNashville__311__Service_Requests.csv')
service_request.head(10)

C:\Users\amand\anaconda3\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,353977,Closed,COVID-19,COVID-19 - Submit Input about The Plan to Reop...,COVID-19 - Submit Input about The Plan to Reop...,04/28/2020 02:33:22 PM,04/28/2020 02:33:22 PM,hubNashville Community,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,354524,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 09:46:53 PM,04/28/2020 09:46:53 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,354188,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 05:09:02 PM,04/28/2020 05:09:02 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,354445,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 08:50:36 PM,04/28/2020 08:50:36 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,334893,Closed,Public Safety,Control Number Request for Towing,NaN,03/29/2020 01:08:55 AM,03/29/2020 01:08:55 AM,hubNashville Community,NaN,NaN,...,False,NaN,True,Long Hollow Pike & Northcreek Blvd,GOODLETTSVILLE,10.0,37072,36.32619,-86.700618,"(36.32619, -86.70061799999999)"
5,334780,Closed,COVID-19,COVID-19 Questions,NaN,03/28/2020 02:17:51 PM,03/28/2020 11:35:56 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,323093,Closed,"Streets, Roads & Sidewalks",Illegal Dumping,Roadway,03/09/2020 01:49:36 PM,03/16/2020 09:13:59 PM,Phone,NaN,NaN,...,False,NaN,False,2600 Grovesnor Dr,ANTIOCH,28.0,37013,36.07597,-86.649030,"(36.07597, -86.64903)"
7,294849,In Progress,"Streets, Roads & Sidewalks",Tree Maintenance on right of way,Tree fell in road or on sidewalk,01/17/2020 04:39:26 PM,NaN,Phone,NaN,NaN,...,False,NaN,False,2419 Kimberly Dr,NASHVILLE,15.0,37214,36.20645,-86.682730,"(36.20645, -86.68273)"
8,323303,Closed,"Trash, Recycling & Litter",Missed Pickup - Recycle Cart Service,Recycle - Curbside/Alley Missed Pickup,03/09/2020 04:26:34 PM,03/17/2020 09:12:24 PM,Phone,NaN,NaN,...,False,NaN,False,1113 Holly St,NASHVILLE,6.0,37206,36.17630,-86.748820,"(36.1763, -86.74882)"
9,295257,Assigned,Public Safety,Abandoned Vehicles,Integration Issue,01/18/2020 09:42:38 PM,NaN,hubNashville Community,NaN,NaN,...,False,NaN,False,2150 Brookview Dr,NASHVILLE,15.0,37214,36.17590,-86.694480,"(36.1759, -86.69448)"


In [111]:
service_request.describe()

,Request #,Preferred Language,Parent Request,Council District,Latitude,Longitude
count,348671.000000,0.0,3589.000000,280574.000000,284188.000000,284188.000000
mean,186628.614992,NaN,188931.867094,17.221286,36.146743,-86.750747
std,104830.612171,NaN,90820.113442,9.517280,0.286789,0.845077
min,144.000000,NaN,14.000000,1.000000,0.000000,-149.900278
25%,96051.500000,NaN,127610.000000,8.000000,36.099950,-86.809932
50%,187781.000000,NaN,196790.000000,18.000000,36.151639,-86.757857
75%,277404.500000,NaN,252934.000000,24.000000,36.187589,-86.707015
max,366272.000000,NaN,366096.000000,35.000000,61.218056,80.072100


In [112]:
service_request['Request Origin'].unique()

array(['hubNashville Community', 'Phone', 'Mobile App', "Mayor's Office",
       "Council's Website", 'In Person at Event', 'Email',
       'Council Member', "Mayor's Website", 'OEM Case', 'Walk-In',
       'Social Media', 'USPS Letter'], dtype=object)

In [113]:
service_request['Request Origin'].value_counts()

Phone                     244656
hubNashville Community     88649
Mobile App                 13051
Email                       1721
Mayor's Office               372
Council Member               129
Council's Website             50
Walk-In                       18
Mayor's Website                8
In Person at Event             6
Social Media                   5
USPS Letter                    5
OEM Case                       1
Name: Request Origin, dtype: int64

In [114]:
list(service_request.columns)

['Request #',
 'Status',
 'Request Type',
 'Subrequest Type',
 'Additional Subrequest Type',
 'Date / Time Opened',
 'Date / Time Closed',
 'Request Origin',
 'System of Origin ID',
 'Contact Type',
 'Preferred Language',
 'State Issue',
 'Parent Request',
 'Closed When Created',
 'Address',
 'City',
 'Council District',
 'ZIP',
 'Latitude',
 'Longitude',
 'Mapped Location']

In [115]:
service_request = service_request[['Status', 'Request Type', 'Subrequest Type',
                                     'Additional Subrequest Type', 'Date / Time Opened', 
                                     'Date / Time Closed', 'Request Origin', 'Parent Request', 
                                     'Address', 'City', 'Council District', 'ZIP', 'Latitude', 
                                     'Longitude']]
service_request.head()

,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,Parent Request,Address,City,Council District,ZIP,Latitude,Longitude
0,Closed,COVID-19,COVID-19 - Submit Input about The Plan to Reop...,COVID-19 - Submit Input about The Plan to Reop...,04/28/2020 02:33:22 PM,04/28/2020 02:33:22 PM,hubNashville Community,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 09:46:53 PM,04/28/2020 09:46:53 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 05:09:02 PM,04/28/2020 05:09:02 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 08:50:36 PM,04/28/2020 08:50:36 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Closed,Public Safety,Control Number Request for Towing,NaN,03/29/2020 01:08:55 AM,03/29/2020 01:08:55 AM,hubNashville Community,NaN,Long Hollow Pike & Northcreek Blvd,GOODLETTSVILLE,10.0,37072,36.32619,-86.700618


In [116]:
service_request = service_request.rename(columns={'Status': 'status', 'Request Type': 'request_type',
                                                   'Subrequest Type': 'subrequest_type', 
                                                    'Additional Subrequest Type': 'additional_subrequest_type', 
                                                    'Date / Time Opened': 'date_time_opened', 
                                                    'Date / Time Closed': 'date_time_closed', 
                                                    'Request Origin' : 'request_origin', 
                                                    'Parent Request': 'parent_request', 'Address': 'address', 
                                                    'City': 'city', 'Council District': 'council_district', 'ZIP': 'zip', 
                                                    'Latitude': 'latitude', 'Longitude': 'longitude'})
service_request.head()

,status,request_type,subrequest_type,additional_subrequest_type,date_time_opened,date_time_closed,request_origin,parent_request,address,city,council_district,zip,latitude,longitude
0,Closed,COVID-19,COVID-19 - Submit Input about The Plan to Reop...,COVID-19 - Submit Input about The Plan to Reop...,04/28/2020 02:33:22 PM,04/28/2020 02:33:22 PM,hubNashville Community,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 09:46:53 PM,04/28/2020 09:46:53 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 05:09:02 PM,04/28/2020 05:09:02 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 08:50:36 PM,04/28/2020 08:50:36 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Closed,Public Safety,Control Number Request for Towing,NaN,03/29/2020 01:08:55 AM,03/29/2020 01:08:55 AM,hubNashville Community,NaN,Long Hollow Pike & Northcreek Blvd,GOODLETTSVILLE,10.0,37072,36.32619,-86.700618


In [117]:
online = ['hubNashville Community', 'Mobile App']
online_req = service_request.query("request_origin in @online")
online_req.head()

,status,request_type,subrequest_type,additional_subrequest_type,date_time_opened,date_time_closed,request_origin,parent_request,address,city,council_district,zip,latitude,longitude
0,Closed,COVID-19,COVID-19 - Submit Input about The Plan to Reop...,COVID-19 - Submit Input about The Plan to Reop...,04/28/2020 02:33:22 PM,04/28/2020 02:33:22 PM,hubNashville Community,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Closed,Public Safety,Control Number Request for Towing,NaN,03/29/2020 01:08:55 AM,03/29/2020 01:08:55 AM,hubNashville Community,NaN,Long Hollow Pike & Northcreek Blvd,GOODLETTSVILLE,10.0,37072,36.326190,-86.700618
5,Closed,COVID-19,COVID-19 Questions,NaN,03/28/2020 02:17:51 PM,03/28/2020 11:35:56 PM,hubNashville Community,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Assigned,Public Safety,Abandoned Vehicles,Integration Issue,01/18/2020 09:42:38 PM,NaN,hubNashville Community,NaN,2150 Brookview Dr,NASHVILLE,15.0,37214,36.175900,-86.694480
10,Closed,"Trash, Recycling & Litter",Cart Service,Recycling - Request an Additional Cart,03/08/2020 09:06:42 PM,03/17/2020 05:13:30 PM,hubNashville Community,NaN,204 Old Williamsburg Ct,NASHVILLE,25.0,37215,36.092855,-86.803188


In [118]:
not_online = ['Phone', "Mayor's Office",
       "Council's Website", 'In Person at Event', 'Email',
       'Council Member', "Mayor's Website", 'OEM Case', 'Walk-In',
       'Social Media', 'USPS Letter']
not_online_req = service_request.query("request_origin in @not_online")
not_online_req.head()

,status,request_type,subrequest_type,additional_subrequest_type,date_time_opened,date_time_closed,request_origin,parent_request,address,city,council_district,zip,latitude,longitude
1,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 09:46:53 PM,04/28/2020 09:46:53 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 05:09:02 PM,04/28/2020 05:09:02 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 08:50:36 PM,04/28/2020 08:50:36 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Closed,"Streets, Roads & Sidewalks",Illegal Dumping,Roadway,03/09/2020 01:49:36 PM,03/16/2020 09:13:59 PM,Phone,NaN,2600 Grovesnor Dr,ANTIOCH,28.0,37013,36.07597,-86.64903
7,In Progress,"Streets, Roads & Sidewalks",Tree Maintenance on right of way,Tree fell in road or on sidewalk,01/17/2020 04:39:26 PM,NaN,Phone,NaN,2419 Kimberly Dr,NASHVILLE,15.0,37214,36.20645,-86.68273


In [119]:
not_online_req['request_type'].value_counts()

Trash, Recycling & Litter     110273
Quick Requests                 64659
Streets, Roads & Sidewalks     60349
Property Violations             4453
Electric & Water General        1575
Transit                         1516
COVID-19                        1240
Other                           1050
Public Safety                    878
Permits                          515
Parks                            234
Storm Relief                     116
Planning & Zoning                 87
Education & Libraries             11
Hands On Volunteers                9
Public Records Request             1
Workforce & Jobs                   1
Name: request_type, dtype: int64

In [120]:
online_req['request_type'].value_counts()

Trash, Recycling & Litter     34873
Streets, Roads & Sidewalks    24481
Property Violations           16198
Public Safety                  8855
COVID-19                       4006
Other                          3764
Electric & Water General       2779
Hands On Volunteers            2628
Transit                        2176
Parks                           836
Permits                         807
Planning & Zoning               163
Storm Relief                     51
Education & Libraries            11
Workforce & Jobs                  7
Public Records Request            1
Name: request_type, dtype: int64

In [121]:
service_request['request_type'].value_counts()

Trash, Recycling & Litter     145146
Streets, Roads & Sidewalks     84830
Quick Requests                 64659
Property Violations            20651
Public Safety                   9733
COVID-19                        5246
Other                           4814
Electric & Water General        4354
Transit                         3692
Hands On Volunteers             2637
Permits                         1322
Parks                           1070
Planning & Zoning                250
Storm Relief                     167
Education & Libraries             22
Workforce & Jobs                   8
Public Records Request             2
Name: request_type, dtype: int64

In [122]:
service_request.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348671 entries, 0 to 348670
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   status                      348671 non-null  object 
 1   request_type                348603 non-null  object 
 2   subrequest_type             338024 non-null  object 
 3   additional_subrequest_type  180510 non-null  object 
 4   date_time_opened            348671 non-null  object 
 5   date_time_closed            338424 non-null  object 
 6   request_origin              348671 non-null  object 
 7   parent_request              3589 non-null    float64
 8   address                     285058 non-null  object 
 9   city                        280771 non-null  object 
 10  council_district            280574 non-null  float64
 11  zip                         280675 non-null  object 
 12  latitude                    284188 non-null  float64
 13  longitude     

In [123]:
service_request.head()

,status,request_type,subrequest_type,additional_subrequest_type,date_time_opened,date_time_closed,request_origin,parent_request,address,city,council_district,zip,latitude,longitude
0,Closed,COVID-19,COVID-19 - Submit Input about The Plan to Reop...,COVID-19 - Submit Input about The Plan to Reop...,04/28/2020 02:33:22 PM,04/28/2020 02:33:22 PM,hubNashville Community,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 09:46:53 PM,04/28/2020 09:46:53 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 05:09:02 PM,04/28/2020 05:09:02 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 08:50:36 PM,04/28/2020 08:50:36 PM,Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Closed,Public Safety,Control Number Request for Towing,NaN,03/29/2020 01:08:55 AM,03/29/2020 01:08:55 AM,hubNashville Community,NaN,Long Hollow Pike & Northcreek Blvd,GOODLETTSVILLE,10.0,37072,36.32619,-86.700618
